In [29]:
#%%
#####################################################
#
# APLICAR PCA A DATOS QUE NUNCA HA VISTO
#
#####################################################
# Deben cargarse los archivos
# - preprocessor_cat.joblib
# - pca_pipe_num.joblib
# - pca_metadata.json
# - T_train_final_objetivo.csv
# - csv de tus nuevos datos
# Devolverá T_new_final.csv: el csv de PCA aplicado a los nuevos datos
 
# === Cargar artefactos para inferencia ===
import joblib
import json
import pandas as pd
#####################################################
preprocessor_cat = joblib.load("../preprocessing/preprocessor_cat.joblib")
pca_pipe = joblib.load("../preprocessing/pca_pipe_num.joblib")
with open("../preprocessing/pca_metadata.json", "r") as f:
#####################################################
    meta = json.load(f)
 
cols_num = meta["cols_num"]
cols_cat = meta["cols_cat"]
pc_cols  = meta["pc_cols"]
cat_out_cols = meta["cat_out_cols"]
 
# PCA DE ENTRENAMIENTO CON OBJETIVO
#####################################################
entrenamiento = pd.read_csv("../preprocessing/T_train_final_objetivo.csv")   ### todas las pca, categoricas y objetivo

#####################################################
entrenamiento_pca_objetivo = entrenamiento[pc_cols]

entrenamiento.head(10)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,heating___electric,...,sewer___none,sewer___public/commercial,sewer___septic,waterfront___No,waterfront___Yes,newConstruction___No,newConstruction___Yes,centralAir___No,centralAir___Yes,objetivo
0,-0.276920,-0.642359,-0.393789,-0.097924,-0.151540,0.721599,-0.058089,0.244951,-0.232479,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,166500
1,1.645793,-0.800673,0.459823,0.397383,0.556896,0.110364,0.320271,-0.292049,0.174033,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,292500
2,1.202848,0.213202,0.426680,-0.021795,-0.890292,0.022758,-0.459030,0.054761,-0.344990,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,196500
3,-1.907295,-0.095893,-0.165868,0.975658,-0.456905,-1.204620,-0.732465,0.446719,-0.078462,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,145000
4,-0.279265,-0.559393,-0.302561,0.243915,-0.678462,0.553153,0.105606,0.141858,-0.161993,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,180000
5,-0.987136,0.815575,0.004551,-0.485773,-0.189593,-0.435615,0.773333,-0.217136,-0.298879,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,113000
6,3.168651,1.723295,0.528124,-0.952748,-0.773595,0.612890,1.913887,-1.037920,-0.187173,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,425000
7,-2.275322,-0.995516,-0.364460,-0.232409,0.281433,-0.341631,0.858585,0.051373,-0.400726,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,92500
8,2.006363,0.153186,0.387926,-0.170307,-0.905025,0.123446,0.578589,0.196654,0.749442,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,252659
9,-0.596610,-0.577971,-0.678002,-0.696258,-0.237654,-0.860828,0.947333,0.231621,-0.247988,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,153000


In [30]:
#%%
 
# NUEVOS DATOS
#####################################################
new_df = pd.read_csv("../../data/Saratoga_desconocidos.csv")
 
#####################################################
X_new_num = new_df[cols_num]
X_new_cat = new_df[cols_cat]
 
# Categóricas (mismo encoder, sin re-ajustar)
X_new_cat_proc = preprocessor_cat.transform(X_new_cat)
df_new_cat_encode = pd.DataFrame(X_new_cat_proc, columns=cat_out_cols, index=new_df.index)
 
# Numéricas → (preproc_num + StdScaler + PCA) con el pipeline guardado
T_new = pca_pipe.transform(X_new_num)
T_new_df = pd.DataFrame(T_new, columns=pc_cols, index=new_df.index)
 
# Final
T_new_final = pd.concat([T_new_df, df_new_cat_encode], axis=1)
T_new_final.to_csv("T_new_final.csv",index=False)

In [31]:
#%%
import plotly.express as px
import plotly.graph_objects as go
 
# --- 1) figura base: train coloreado por objetivo (igual que antes) ---
y_train = entrenamiento["objetivo"]
df_plot2 = entrenamiento_pca_objetivo.iloc[:, :2].copy()
df_plot2["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)
 
fig = px.scatter(
    df_plot2, x="PC1", y="PC2", color="objetivo",
    opacity=0.85, title="PC1 vs PC2 — Train (objetivo) + Nuevos",
    height=550
)
fig.update_traces(marker=dict(size=6))
 
# --- 2) overlay: nuevos (un solo trazo, símbolo distinto) ---
new2 = T_new_df.loc[:, ["PC1", "PC2"]].copy()
fig.add_trace(
    go.Scatter(
        x=new2["PC1"], y=new2["PC2"],
        mode="markers",
        name="nuevos",
        marker=dict(size=9, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)
 
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()

In [32]:
T_new_final

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,heating___electric,...,fuel___oil,sewer___none,sewer___public/commercial,sewer___septic,waterfront___No,waterfront___Yes,newConstruction___No,newConstruction___Yes,centralAir___No,centralAir___Yes
0,-1.905764,0.266144,1.022320,0.168308,-0.258120,-0.460221,0.535150,0.658734,-0.065351,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.159719,-0.761804,-0.909746,0.376705,-0.876126,-0.042257,0.401184,-0.496953,0.306853,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,2.898206,-0.079321,-0.177984,-0.608690,-0.464261,-0.411519,-0.478531,-0.768703,0.056403,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1.737947,0.421540,0.292222,-0.020164,-0.664596,0.260617,0.535103,0.279524,0.257682,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.872174,-0.081409,0.664935,0.486500,-0.750387,1.817104,-0.407539,0.576972,0.447735,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,2.604294,0.842840,0.212242,-0.822322,-1.252775,-0.201655,-0.321889,0.262750,-0.461476,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,2.477326,0.845647,-0.011809,0.120092,-0.586127,-0.100111,0.089898,-0.575221,-0.060684,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,-2.868784,-0.813204,-0.410343,0.253822,0.446382,-0.362790,-0.243518,0.251302,0.545581,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8,-1.198604,0.472026,0.376708,-0.123120,-0.461421,1.070770,-0.945298,-0.247469,-0.176919,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9,-1.436986,1.870052,0.903307,-0.068640,0.261806,0.006589,0.228083,-0.149723,0.973143,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
